In [41]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import puzz

In [42]:
net = puzz.Network(['a1', 'a2'])
net.add_obstacle('o1')
net.add_obstacle('o2')
net.add_obstacle('o3')
net.add_obstacle('o4')
net.add_ability('a3')
net.add_ability('a4')
net.nodes

{'a1': <puzz.Ability at 0x106831910>,
 'a2': <puzz.Ability at 0x106831650>,
 'a3': <puzz.Ability at 0x106831bd0>,
 'a4': <puzz.Ability at 0x106831790>,
 'o1': <puzz.Node at 0x106831510>,
 'o2': <puzz.Node at 0x106864ad0>,
 'o3': <puzz.Node at 0x106831cd0>,
 'o4': <puzz.Node at 0x106831f90>,
 'start': <puzz.Node at 0x106831e50>}

In [43]:
net.defeats('a1', 'o1')
net.defeats('a2', 'o2')
net.defeats('a3', 'o3')
net.defeats('a3', 'o1')
net.defeats('a4', 'o4')

In [46]:
net.calc_ability_prereqs()

In [47]:
# desired behavior: a3 obsoletes a1. so you should only be able to
# place it after a1, not a2
net.enabled_nodes('a1')
net.enabled_nodes('a2')

{'a3', 'a4', 'o1', 'o3', 'o4'}

{'a4', 'o1', 'o3', 'o4'}